### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe


### What is LCEL?

LCEL stands for LangChain Expression Language. It is a powerful tool within LangChain that allows developers to chain components together in a structured and efficient manner. LCEL simplifies the process of building complex workflows by enabling seamless integration of language models, prompt templates, and output parsers.

With LCEL, you can:

- Combine multiple components into a single workflow.
- Debug and trace your application effectively using LangSmith.
- Deploy your application easily with LangServe.

LCEL is particularly useful for creating modular and reusable components in your LLM applications.


In [3]:
### Open AI API Key and Open Source Models --Llama2, Gemma, Mistral --Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")



In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model ="gemma2-9b-it", groq_api_key=groq_api_key)
model

ModuleNotFoundError: No module named 'langchain_openai'

In [32]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="Translate the following English text to French"),
    HumanMessage(
        content="Hello, how are you?")
]

response = model.invoke(messages)


In [33]:
response

AIMessage(content='Bonjour, comment allez-vous ? \n\n\nThis is the most common and formal way to say "Hello, how are you?" in French. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 22, 'total_tokens': 55, 'completion_time': 0.06, 'prompt_time': 0.002229266, 'queue_time': 0.233778333, 'total_time': 0.062229266}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c34d75fe-6f05-4223-bf37-1f6b44b13bab-0', usage_metadata={'input_tokens': 22, 'output_tokens': 33, 'total_tokens': 55})

In [34]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

'Bonjour, comment allez-vous ? \n\n\nThis is the most common and formal way to say "Hello, how are you?" in French. \n'

In [35]:
### Using LCEL -- chain the components
chain = model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [37]:
### Prompt Template
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("human", "{text}")] )

In [41]:
result = prompt.invoke({"language": "French", "text": "Hello?"})

In [42]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello?', additional_kwargs={}, response_metadata={})]

In [ ]:
## Chaining together with LCEL 
chain = prompt | model | parser  
chain.invoke({"language": "French", "text": "Hello?"})

"Bonjour ? \n\n\nLet me know if you have any other phrases you'd like translated!\n"